In [ ]:
import pandas as pd
from pandas import json_normalize
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from tqdm import tqdm
import time 

In [ ]:
# loading cluster dataset

In [ ]:
def song_recommender(df_top100, df_total):
    song = input("Please enter the name of a song: ")
    